### 곰스실적분석
1. 계약목록을 통해 일자별 보험료 그래프

In [ ]:
import pandas as pd
#fl='D:/글로벌/실적분석/곰스_2002.xlsx' 
fl='E:/글로벌백업(180329)/04.수수료작업/201912수수료/계약목록.xlsx'
df= pd.read_excel(fl)

df= df[df.NO!='합계']   #불필요 데이터 삭제
df['지점'].fillna('엠피메트로', inplace=True)  #결측치 보정

#지점명 변환 join
fl='D:/글로벌/실적분석/코드파일.xlsx'   
fl='E:/글로벌백업(180329)/04.수수료작업/코드목록.xlsx'
df_org= pd.read_excel(fl,sheet_name='지점변환')
df=df.join(df_org.set_index('수금사원명')['변환지점'], on='수금사원명')  
df['변환지점']=df['변환지점'].fillna(df.지점)
df=df.rename(columns={'지점':'원지점','변환지점':'지점'}) 

In [ ]:
import datetime
fr=pd.to_datetime('2019-01-01',format='%Y-%m-%d')
to=pd.to_datetime('2020-02-29',format='%Y-%m-%d')
df.계약일자=pd.to_datetime(df.계약일자,format='%Y-%m-%d')
cond1= (df.계약일자 >= fr) & (df.계약일자 <= to)
df=df[cond1]

In [ ]:
fl='D:/글로벌/실적분석/코드파일.xlsx'
fl='E:/글로벌백업(180329)/04.수수료작업/코드목록.xlsx'
df_grp= pd.read_excel(fl,sheet_name='지점그룹')
#df.join(df_grp.set_index('지점')['지점그룹'], on='지점') 
df=pd.merge(df,df_grp,how='left',on='지점')

In [ ]:
import matplotlib.pyplot as plt

grp='직영'      #option 1. 직영 or 지사
graphs='누적'   #optiom 2. 누적 or 일자별
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (12, 8)
plt.figure()

plt.ylabel('보험료')
plt.xlabel('일자')
plt.title(grp+' 일자별 실적추세' + '-' + graphs)

df2=df.query("(지점그룹=='{}') and (계약종류 == ['{}','{}'])".format(grp,'장기','생보'))
df2=pd.DataFrame(df2.groupby(['지점그룹','지점','계약일자'])['초회보험료'].agg('sum'))
df2.reset_index(inplace=True)
brhs=df2.지점.unique()

for brh in brhs:
    dfTemp=df2[df2.지점==brh][['계약일자','초회보험료']].set_index('계약일자')
    if graphs=='누적': 
        plt.plot(dfTemp.cumsum()/1000,label=brh)
    else:   plt.plot(dfTemp,label=brh)
plt.legend(loc=0)
plt.savefig('D:/글로벌/실적분석/ss.png')
plt.show()


### 구성비 및 지점별 실적  
 pie, barh 그래프

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

st='정상'      #option 1. 직영 or 지사
graphs='누적'   #optiom 2. 누적 or 일자별
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (12, 8)
fig= plt.figure()

ax1 = fig.add_subplot(1, 2, 1) 
ax2 = fig.add_subplot(1, 2, 2) 
plt.subplots_adjust(wspace=1.1)
                    

df2=df.query("(계약상태=='{}') and (계약종류 == ['{}','{}'])".format(st,'장기','생보'))


# Pie chart, where the slices will be ordered and plotted counter-clockwise:
explode = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a c

dfT=pd.DataFrame(df2.groupby('지점그룹')['초회보험료'].agg('sum')/1000)
lbls=dfT.index

def func(pct, allvals):
    absolute = int(pct/1000.*np.sum(allvals))
    return "{:.1f}%\n({:,d}천)".format(pct, absolute)

wedges, texts, autotexts = ax1.pie(dfT, autopct=lambda pct: func(pct, dfT),
                                  textprops=dict(color="w"),explode=explode )

ax1.legend(wedges, lbls,
          title="Ratio",
          loc="upper left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.setp(autotexts, size=16, weight="bold")
ax1.set_title("Group: 구성비")

#---bar차트
plt.ylabel('지점')
plt.xlabel('실적')
plt.title(' 지점별 실적' + '-' + graphs)


dfT=pd.DataFrame(df2[df2.지점그룹=='지사'].groupby('지점')['초회보험료'].agg('sum')).reset_index()
val=dfT.초회보험료/1000 
cnt=len(dfT.초회보험료)  # y축 위치값 => 2번째 그래프에서 사용
for y, x in enumerate(val):
    str='{:,d}'.format(int(x))
    ax2.text(x,y,str,va='center' )           #막대에 값 표시(x,y,값)
ax2.barh(dfT.지점,dfT.초회보험료/1000)
ax2.text

dfT=pd.DataFrame(df2[df2.지점그룹=='직영'].groupby('지점')['초회보험료'].agg('sum')).reset_index()
val=dfT.초회보험료/1000 
for y, x in enumerate(val):
    str='{:,d}'.format(int(x))
    ax2.text(x,cnt+y,str,va='center' )   #y축위치츷 첫번째 그래프 위치를 더해서 시작함

ax2.barh(dfT.지점,dfT.초회보험료/1000)
plt.tight_layout()
plt.show()

### 평균대비 차이 실적  
 bar , subplot별 속성  

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

st='정상'      #option 1. 직영 or 지사
graphs='차이'   #optiom 2. 누적 or 일자별
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (20, 5)
fig= plt.figure()

ax1 = fig.add_subplot(1, 2, 1) 
ax2 = fig.add_subplot(1, 2, 2) 
#plt.subplots_adjust(wspace=1.0)
                    
df2=df.query("(계약상태=='{}') and (계약종류 == ['{}','{}'])".format(st,'장기','생보'))

#---bar차트
#1번 그래프
dfT=pd.DataFrame(df2[df2.지점그룹=='지사'].groupby('지점')['초회보험료'].agg('sum')).reset_index()
dfT.초회보험료=dfT.초회보험료/1000
mean=int(dfT.초회보험료.mean())
dfT['차이']= (dfT.초회보험료 - mean ) 

val=dfT.차이
cnt=len(dfT.차이)  # y축 위치값 => 2번째 그래프에서 사용
for x, y in enumerate(val):
    str='{:,d}'.format(int(y))
    ax1.text(x,y,str,ha='center' )           #막대에 값 표시(x,y,값)
ax1.bar(dfT.지점,dfT.차이,color='ry')
ax1.axhline(0,color='k')
ax1.legend('',title='지사평균={:,d}'.format(mean),loc=2,fontsize=20)
ax1.set_xlabel("지사")
ax1.set_ylabel("Diff")
ax1.set_title("지사 평균대비 差")

#2번 그래프
dfT=pd.DataFrame(df2[df2.지점그룹=='직영'].groupby('지점')['초회보험료'].agg('sum')).reset_index()
dfT.초회보험료=dfT.초회보험료/1000
mean=int(dfT.초회보험료.mean())
dfT['차이']= (dfT.초회보험료 - mean ) 
val=dfT.차이
for x, y in enumerate(val):
    str='{:,d}'.format(int(y))
    ax2.text(x,cnt+y,str,ha='center' )   #y축위치츷 첫번째 그래프 위치를 더해서 시작함

ax2.bar(dfT.지점,dfT.차이)
ax2.axhline(0,color='k')
ax2.legend('',title='직영평균={:,d}'.format(mean),loc=2,fontsize=20)
ax2.set_xlabel("지점")
ax2.set_ylabel("Diff")
ax2.set_title("직영 평균대비 差")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

st='정상'      #option 1. 직영 or 지사
graphs='누적'   #optiom 2. 누적 or 일자별
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (12, 8)
fig= plt.figure()

ax1 = fig.add_subplot(1, 2, 1) 
ax2 = fig.add_subplot(1, 2, 2) 
plt.subplots_adjust(wspace=1.1)
                    

df2=df.query("(계약상태=='{}') and (계약종류 == ['{}','{}'])".format(st,'장기','생보'))


#---bar차트
plt.ylabel('지점')
plt.xlabel('실적')
plt.title(' 지점별 실적' + '-' + graphs)


dfT=pd.DataFrame(df2[df2.지점그룹=='지사'].groupby('지점')['초회보험료'].agg('sum')).reset_index()

val=dfT.초회보험료/1000 
cnt=len(dfT.초회보험료)  # y축 위치값 => 2번째 그래프에서 사용
for y, x in enumerate(val):
    str='{:,d}'.format(int(x))
    ax2.text(x,y,str,va='center' )           #막대에 값 표시(x,y,값)
ax2.bar(dfT.지점,dfT.초회보험료/1000)
ax2.text

dfT=pd.DataFrame(df2[df2.지점그룹=='직영'].groupby('지점')['초회보험료'].agg('sum')).reset_index()
val=dfT.초회보험료/1000 
for y, x in enumerate(val):
    str='{:,d}'.format(int(x))
    ax2.text(x,cnt+y,str,va='center' )   #y축위치츷 첫번째 그래프 위치를 더해서 시작함

ax2.bar(dfT.지점,dfT.초회보험료/1000)
plt.tight_layout()
plt.show()

In [ ]:
df.rename(columns={'지점그룹_x':'지점그룹'}, inplace=True)

In [ ]:
df2=pd.pivot_table(df,index=['지점그룹','지점'],columns=['계약종류'],values='초회보험료',
                   aggfunc=[np.sum,np.mean],fill_value=0)

In [ ]:
df2.columns=['합','평']

In [ ]:
#컬럼명 변경
df2.columns=[n[0]+n[1].replace() for n in df2.columns]

In [ ]:
df_m=pd.melt(df2,id_vars=['지점그룹','지점'],value_vars='sum생보')
df_m.head(2)

In [5]:
import pandas as pd
sales=pd.read_csv('../testData/sales.csv')

In [6]:
sales.head()

,date,dept,code,qty,cost
0,2018-01-01,a1,c1,10,1
1,2018-01-02,a2,c2,20,2
2,2018-01-03,a1,c3,21,14
3,2018-01-04,a2,c2,22,14
4,2018-01-05,a3,c2,23,14


In [ ]:
#sales.pivot(index='date',columns='code')
pd.pivot_table?

In [23]:
pd.pivot_table(sales,index=['date','dept'],columns='code',values=['qty','cost'],aggfunc=['sum','mean'],fill_value=0)

sum                             mean                          \
                cost             qty             cost             qty           
code              c1  c2  c3  c4  c1  c2  c3  c4   c1  c2  c3  c4  c1  c2  c3   
date       dept                                                                 
2018-01-01 a1      1   0   0   0  10   0   0   0    1   0   0   0  10   0   0   
           a3      0   0   0  20   0   0   0  10    0   0   0  20   0   0   0   
2018-01-02 a1      0   0   0  10   0   0   0  20    0   0   0  10   0   0   0   
           a2      0   2   0   0   0  20   0   0    0   2   0   0   0  20   0   
2018-01-03 a1      0   0  14   0   0   0  21   0    0   0  14   0   0   0  21   
           a2      0   0   0  11   0   0   0  21    0   0   0  11   0   0   0   
2018-01-04 a1      0   0   0  12   0   0   0  22    0   0   0  12   0   0   0   
           a2      0  14   0   0   0  22   0   0    0  14   0   0   0  22   0   
2018-01-05 a2      0   0   0  12   0   0   0  23    0   0   0  12   0   0   0   
           a3      0  14   0   0   0  23   0   0    0  14   0   0   0  23   0   
2018-01-06 a3     14   0   0  13  21   0   0  21   14   0   0  13  21   0   0   
2018-01-07 a1      0  14   0   0   0  22   0   0    0  14   0   0   0  22   0   
           a3      0   0   0  14   0   0   0  22    0   0   0  14   0   0   0   
2018-01-08 a2      0   0  14  15   0   0  26  26    0   0  14  15   0   0  26   
2018-01-09 a1     14   0   0   0  27   0   0   0   14   0   0   0  27   0   0   
           a3      0   0   0  16   0   0   0  27    0   0   0  16   0   0   0   
2018-01-10 a2      0  10   0   0   0  28   0   0    0  10   0   0   0  28   0   
           a3      0   0   0  17   0   0   0  28    0   0   0  17   0   0   0   
2018-01-11 a3      0   0  11  18   0   0  29  29    0   0  11  18   0   0  29   
2018-01-12 a3     12   0   0  19  30   0   0  30   12   0   0  19  30   0   0   
2018-01-13 a1      0   0   0  20   0   0   0  30    0   0   0  20   0   0   0   
           a2      0  12   0   0   0  30   0   0    0  12   0   0   0  30   0   
2018-01-14 a2      0   0   0  17   0   0   0  21    0   0   0  17   0   0   0   
           a3      0   0  13   0   0   0  21   0    0   0  13   0   0   0  21   
2018-01-15 a1      0   0   0  18   0   0   0  22    0   0   0  18   0   0   0   
           a3     14   0   0   0  22   0   0   0   14   0   0   0  22   0   0   
2018-01-16 a1      0  15   0   0   0  26   0   0    0  15   0   0   0  26   0   
           a2      0   0   0  19   0   0   0  26    0   0   0  19   0   0   0   
2018-01-17 a1      0   0   0  20   0   0   0  27    0   0   0  20   0   0   0   
           a2      0   0  16   0   0   0  27   0    0   0  16   0   0   0  27   
2018-01-18 a1     17   0   0   0  28   0   0   0   17   0   0   0  28   0   0   
           a2      0   0   0  17   0   0   0  28    0   0   0  17   0   0   0   
2018-01-19 a1      0   0   0  18   0   0   0  29    0   0   0  18   0   0   0   
           a2      0  18   0   0   0  29   0   0    0  18   0   0   0  29   0   
2018-01-20 a2      0   0   0  19   0   0   0  30    0   0   0  19   0   0   0   
           a3      0   0  19   0   0   0  30   0    0   0  19   0   0   0  30   

                     
                     
code             c4  
date       dept      
2018-01-01 a1     0  
           a3    10  
2018-01-02 a1    20  
           a2     0  
2018-01-03 a1     0  
           a2    21  
2018-01-04 a1    22  
           a2     0  
2018-01-05 a2    23  
           a3     0  
2018-01-06 a3    21  
2018-01-07 a1     0  
           a3    22  
2018-01-08 a2    26  
2018-01-09 a1     0  
           a3    27  
2018-01-10 a2     0  
           a3    28  
2018-01-11 a3    29  
2018-01-12 a3    30  
2018-01-13 a1    30  
           a2     0  
2018-01-14 a2    21  
           a3     0  
2018-01-15 a1    22  
           a3     0  
2018-01-16 a1     0  
           a2    26  
2018-01-17 a1    27  
           a2     0  
2018-01-18 a1     0  
      

In [ ]:
# pd.DataFrame( )